In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import the dataset 
# Medical Cost Personal Dataset
dataset = pd.read_csv('/kaggle/input/insurance/insurance.csv')

**Let us quickly check what features and label do we have**

In [ ]:
dataset.head()

**We have few information about each beneficiary like what is his/her age,if that person has children or not,if he/she is a smoker etc.The very last column is our label vector.**

**We need to check if we have NaN in the dataset.If there is any missing data then we need to take care of theat.**

In [ ]:
dataset.isnull().sum()

**That is quite impressive.We do not have any missing data in the dataset.So we don't need to worry about that.**

**We have three features which have string values.Ofcourse we cannot feed them directly to our model.**
To take care of these categorical features we will encode them:)

In [ ]:
# Encode categorical data
# Create instance
encode = LabelEncoder()

# Gender of the benificiary
encode.fit(dataset.sex.drop_duplicates()) 
dataset.sex = encode.transform(dataset.sex)

# Is the benificiary smoker or non-smoker
encode.fit(dataset.smoker.drop_duplicates()) 
dataset.smoker = encode.transform(dataset.smoker)

# Residence of benificiary
encode.fit(dataset.region.drop_duplicates()) 
dataset.region = encode.transform(dataset.region)

**Let us check how our dataset looks like now:)**

In [ ]:
dataset.head()

**Now let's visualize correlation between these features and label.**

In [ ]:
dataset.corr()['charges'].sort_values()
f, ax = plt.subplots(figsize=(10, 8))
corr = dataset.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap='Greens',
            square=True, ax=ax)

**Strong correlation between smoking and higher charges as expected.**

**Let us have some more fun with visualization part:)**

In [ ]:
plt.figure(figsize=(12,7))
plt.title("Distribution")
sns.distplot(dataset,color='r')

In [ ]:
sns.pairplot(dataset)

We already noticed there is some strong relation of smokers and people who have higher charges.Let us do some more research about this.

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'sex', palette="pastel", data=dataset)

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'children', palette="rainbow", data=dataset)

**Distribution**

In [ ]:
plt.figure(figsize=(12,7))
plt.title("BMI")
sns.distplot(dataset["bmi"], color = 'c')

In [ ]:
plt.figure(figsize=(12,7))
plt.title("Children")
sns.distplot(dataset["children"], color = 'r')

In [ ]:
plt.figure(figsize=(12,7))
plt.title("Age")
sns.distplot(dataset["age"], color = 'g')

In [ ]:
sns.catplot(x="children", kind="count", palette="rainbow", data=dataset)

**Time to create feature matrix and label vector:)**

In [ ]:
# Get feature matrix and label vector
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

**Let us split our dataset into training set and test set.**

The size of our training set is 80% of the whole set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Linear Regression Model**

In [ ]:
# Train the Multiple Linear Regression model on the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predict the Test set results
y_pred = regressor.predict(X_test)

In [ ]:
print(regressor.score(X_test,y_test))

Well not that bad!

**Decision Tree Model**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state = 0)
dtr.fit(X_train, y_train)
dtr_y_pred = dtr.predict(X_test)

In [ ]:
print('MSE test data: %.3f' % (mean_squared_error(y_test,dtr_y_pred)))
print('R2 test data: %.3f' % (r2_score(y_test,dtr_y_pred)))

Not that impressive:(

**Random Forest Model**

Hope this one will give us better result:)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 10, random_state = 0)
rfr.fit(X_train, y_train)
rfr_y_pred = rfr.predict(X_test)

In [ ]:
print('MSE test data: %.3f' % (mean_squared_error(y_test,rfr_y_pred)))
print('R2 test data: %.3f' % (r2_score(y_test,rfr_y_pred)))

Yes!!

**Visualization**

In [ ]:
plt.figure(figsize=(12,7))
plt.title("FINAL PREDICTION")
plt.scatter(rfr.predict(X_train),rfr.predict(X_train)- y_train,c = 'red', marker = 'o', s = 30, alpha = 0.6,label = 'Train data')
plt.scatter(rfr_y_pred,rfr_y_pred - y_test,c = 'cyan', marker = 'o', s = 30, alpha = 0.8,label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend()
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'black')
plt.show()

**If you like this then please consider upvoting!**

**Thank you!**